In [5]:
import pandas as pd
import math

# --- Equation functions ---

def loglogw1(a, b, x, mse=0): return math.exp(a + b * math.log(math.log(x + 1)) + (mse / 2))
def loglogw2(a, b, x, mse=0): return math.exp(a + b * math.log(math.log(x + 1)) + (math.sqrt(x) * (mse / 2)))
def loglogw3(a, b, x, mse=0): return math.exp(a + b * math.log(math.log(x + 1)) + x * (mse / 2))
def loglogw4(a, b, x, mse=0): return math.exp(a + b * math.log(math.log(x + 1)) + (x ** 2) * (mse / 2))
def lin(a, b, x): return a + b * x
def quad(a, b, c, x): return a + b * x + c * x ** 2
def cub(a, b, c, d, x): return a + b * x + c * x ** 2 + d * x ** 3
def quart(a, b, c, d, e, x): return a + b * x + c * x ** 2 + d * x ** 3 + e * x ** 4
def expow1(a, b, x, mse=0): return math.exp(a + b * x + (mse / 2))
def expow2(a, b, x, mse=0): return math.exp(a + b * x + math.sqrt(x) * (mse / 2))
def expow3(a, b, x, mse=0): return math.exp(a + b * x + x * (mse / 2))
def expow4(a, b, x, mse=0): return math.exp(a + b * x + (x ** 2) * (mse / 2))

eq_funcs = {
    "lin": lin,
    "quad": quad,
    "cub": cub,
    "quart": quart,
    "loglogw1": loglogw1,
    "loglogw2": loglogw2,
    "loglogw3": loglogw3,
    "loglogw4": loglogw4,
    "expow1": expow1,
    "expow2": expow2,
    "expow3": expow3,
    "expow4": expow4
}

# --- Load and filter coefficients ---

df = pd.read_csv("TS6_Growth_coefficients.csv")
fraxinus_df = df[df['Scientific Name'].str.contains("fraxinus excelsior", case=False)].copy()

# Keep only needed columns
param_cols = ['Independent variable', 'Predicts component ', 'EqName', 'a', 'b', 'c', 'd', 'e']
fraxinus_df = fraxinus_df[param_cols]

# --- Compute growth database for age 0–50 ---

data_rows = []
for age in range(0, 51):
    values = {"age": age}

    # Step 1: Find dbh from age
    dbh_row = fraxinus_df[
        (fraxinus_df['Independent variable'].str.strip() == 'age') &
        (fraxinus_df['Predicts component '].str.strip() == 'dbh')
    ].iloc[0]

    eqname = dbh_row['EqName']
    a, b, c, d, e = [dbh_row.get(k, 0) for k in ['a', 'b', 'c', 'd', 'e']]
    dbh = eq_funcs[eqname](a, b, age) if eqname == "lin" else eq_funcs[eqname](a, b, c, age)
    values["dbh"] = dbh

    # Step 2: Use dbh to calculate all others
    for _, row in fraxinus_df.iterrows():
        indep = row['Independent variable'].strip()
        dep = row['Predicts component '].strip()

        if indep != "dbh" or dep == "dbh" or dep == "age":
            continue

        eqname = row['EqName']
        a, b, c, d, e = [row.get(k, 0) for k in ['a', 'b', 'c', 'd', 'e']]

        if indep == "dbh":
            try:
                if eqname in ["lin", "loglogw1", "loglogw2", "loglogw3", "loglogw4", "expow1", "expow2", "expow3", "expow4"]:
                    val = eq_funcs[eqname](a, b, dbh)
                elif eqname == "quad":
                    val = quad(a, b, c, dbh)
                elif eqname == "cub":
                    val = cub(a, b, c, d, dbh)
                elif eqname == "quart":
                    val = quart(a, b, c, d, e, dbh)
                else:
                    val = None
                values[dep] = val
            except:
                values[dep] = None

    data_rows.append(values)

# --- Create DataFrame and export ---
result_df = pd.DataFrame(data_rows)
result_df.to_json("fraxinus_excelsior_database.json", orient="records", indent=2)
print(result_df.head())


   age      dbh  crown dia  crown ht  leaf area   tree ht
0    0  3.06926   0.990379  2.346749   1.840843  3.112917
1    1  4.18978   1.369401  2.642132   3.878624  3.622662
2    2  5.31030   1.742271  2.930458   6.544215  4.105017
3    3  6.43082   2.108988  3.211728   9.727676  4.560911
4    4  7.55134   2.469553  3.485942  13.337580  4.991272


Fraxinus excelsior falls in the large dedicious category, for this according to Table 8—Percentage frequency of best-fit equation forms by measured/predicted parameter and tree type choose best models for each parameter:
| Relationship                  | cub | expow | lin | log-log | quad | quart |
|------------------------------|-----|--------|-----|----------|------|--------|
| Age to d.b.h.                | 33  | 0      | 24  | 9        | 34   | 0      |
| Crown diameter to d.b.h.     | 35  | 1      | 11  | 35       | 19   | 0      |
| D.b.h. to age                | 43  | 1      | 23  | 7        | 26   | 0      |
| D.b.h. to crown diameter     | 29  | 0      | 6   | 31       | 34   | 0      |
| D.b.h. to crown height       | 13  | 0      | 17  | 29       | 40   | 0      |
| D.b.h. to leaf area          | 5   | 1      | 1   | 85       | 8    | 0      |
| D.b.h. to tree height        | 18  | 0      | 10  | 33       | 39   | 0      |